In [3]:
# import libraries
import pandas as pd
from geopy import distance as dist

In [4]:
# constants
RADIUS = 100 # metres, approx. 330ft
ARRIVAL_SPEED = 20 # kph, approx 12.5 mph

In [5]:
# fetch data from csv
data = pd.read_csv("Heartbeats.csv")
data

,user_id,user_role,timestamp,latitude,longitude,speed
0,7,driver,1669980219,18.0088,-76.7823,40
1,7,driver,1669980224,18.0090,-76.7822,40
2,7,driver,1669980229,18.0093,-76.7822,40
3,7,driver,1669980234,18.0096,-76.7820,40
4,7,driver,1669980239,18.0096,-76.7820,40
...,...,...,...,...,...,...
353,7,driver,1669981984,18.0054,-76.7477,40
354,7,driver,1669981989,18.0056,-76.7475,40
355,7,driver,1669981994,18.0058,-76.7474,32
356,7,driver,1669981999,18.0059,-76.7473,17


In [6]:
# list of pickup points selected at random
locations = [57, 102, 168, 212, 281, 348, data.shape[0] - 1]
pickup_points = [];
for location in locations:
    pickup_points.append(data.loc[location, "latitude" : "longitude"]) 

In [7]:
# params: current_loc (data frame obj), dest (data frame obj)
def get_distance(curr_loc, dest): 
    # tuple(lat, lon)
    point_one = (curr_loc[0], curr_loc[1])
    point_two = (dest[0], dest[1])
    
    # calculate current distance in metres 
    return dist.great_circle(point_one, point_two).m

In [8]:
def main_no_speed():
    # heartbeat counter, passenger counter
    i, j = 0, 1
    
    # initialize prev_dist to inf  
    prev_dist = float("inf")
    
    while len(pickup_points) and i < data.shape[0]:
        # get curr_dist         
        curr_dist = get_distance(data.loc[i, "latitude":"longitude"], pickup_points[0])
        
        # not important -> only printing when approaching pickup point         
        if prev_dist > curr_dist:
            print("Index {}: Enroute to pickup point #{}. current distance: {:0.2f}km.".format(i, j, curr_dist / 1000))
        
        # driver is assumed to arrive when curr_dist < 100m 
        if curr_dist < RADIUS:
            if prev_dist < curr_dist:
                print("✅ Index {}: Picked up passenger(s), enroute to pickup point #{}.".format(i, j + 1))
                
                # dequeue current pickup point
                pickup_points.pop(0)
                j += 1
            else:
                print("🎯 Index {}: Within {:0.2f}m of pickup point #{}.".format(i, curr_dist, j))
        
        # set prev_dist to curr_dist while approaching pickup point, otherwise reset it to inf
        if prev_dist > curr_dist:
            prev_dist = curr_dist
        else:
            prev_dist = float("inf")
        
        # increment counter by 1
        i += 1

In [9]:
def main_with_speed():
    # heartbeat counter, passenger counter
    i, j = 0, 1

    while len(pickup_points) and i < data.shape[0]:
        # get curr_dist
        curr_dist = get_distance(data.loc[i, "latitude":"longitude"], pickup_points[0])
        speed = data.loc[i, "speed"]

        print("Index {}: Enroute to pickup point #{}. current distance: {:0.2f}km, current speed: {}kph.".format(i, j, curr_dist / 1000, speed))

        # driver is considered close when curr_dist < 100m and speed < 20kph
        if curr_dist <= RADIUS and speed <= ARRIVAL_SPEED:
            if speed == 0:
                print("✅ Index {}: Picked up passenger(s), enroute to pickup point #{}.".format(i, j + 1))

                # dequeue current pickup point
                pickup_points.pop(0)
                j += 1
            else:
                print("🎯 Index {}: Within {:0.2f}m of pickup point #{}, current speed: {}kph.".format(i, curr_dist, j, speed))
        i += 1

main_with_speed()

Index 0: Enroute to pickup point #1. current distance: 1.90km, current speed: 40kph.
Index 1: Enroute to pickup point #1. current distance: 1.89km, current speed: 40kph.
Index 2: Enroute to pickup point #1. current distance: 1.86km, current speed: 40kph.
Index 3: Enroute to pickup point #1. current distance: 1.84km, current speed: 40kph.
Index 4: Enroute to pickup point #1. current distance: 1.84km, current speed: 40kph.
Index 5: Enroute to pickup point #1. current distance: 1.86km, current speed: 40kph.
Index 6: Enroute to pickup point #1. current distance: 1.90km, current speed: 40kph.
Index 7: Enroute to pickup point #1. current distance: 1.92km, current speed: 40kph.
Index 8: Enroute to pickup point #1. current distance: 1.94km, current speed: 40kph.
Index 9: Enroute to pickup point #1. current distance: 2.00km, current speed: 40kph.
Index 10: Enroute to pickup point #1. current distance: 2.04km, current speed: 40kph.
Index 11: Enroute to pickup point #1. current distance: 2.04km, 